# Лабараторна робота №2 
# Наука про дані: підготовчий етап
## Невмержицька Дар'я ФБ-23
### Мета роботи: ознайомитися з основними кроками по роботі з даними – workflow від постановки задачі до написання пояснювальної записки, зрозуміти постановку задачі та природу даних, над якими виконується аналітичні операції
### Хід роботи:
#### 1)Створити env в якому будуть встановлені всі необхідні бібліотеки та налаштування для данної лабораторної роботи

In [1]:
import urllib.request
import pandas as pd
import os
import ssl
from datetime import datetime

#### 2) Для кожної із адміністративних одиниць України завантажити тестові труктуровані файли, що містять значення VHI-індексу. Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження;

In [2]:
def download(index):
    # Формуємо URL для завантаження даних VHI-індексу для заданої адміністративної одиниці
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?provinceID={index}&country=UKR&yearlyTag=Weekly&type=Mean&TagCropland=crop&year1=1982&year2=2024"
    
    # Отримуємо поточну дату та час
    now = datetime.now()
    date_time_now = now.strftime("%d-%m-%Y_%H-%M-%S")
    
    # Вимикаємо перевірку сертифікатів SSL
    context = ssl.create_default_context()
    context.check_hostname = False
    context.verify_mode = ssl.CERT_NONE
    
    
    # Видаляємо попередні файли, якщо вони існують
    previous_files = [f for f in os.listdir("AD") if f.startswith(f"NOAA_ID_{index}_")]
    for file in previous_files:
        os.remove(os.path.join("AD", file))
    
    # Відкриваємо URL для читання
    with urllib.request.urlopen(url, context=context) as vhi_url:
        # Створюємо файл для запису з іменем, що містить індекс та дату та час завантаження
        with open(f"AD/NOAA_ID_{index}_{date_time_now}.csv", 'wb') as out:
            # Записуємо дані у файл
            out.write(vhi_url.read())
    
    print(f"Файл завантажено NOAA_ID_{index}_{date_time_now}.csv")

for i in range(1, 28):
    download(i)


Файл завантажено NOAA_ID_1_23-03-2024_08-02-57.csv
Файл завантажено NOAA_ID_2_23-03-2024_08-03-01.csv
Файл завантажено NOAA_ID_3_23-03-2024_08-03-02.csv
Файл завантажено NOAA_ID_4_23-03-2024_08-03-04.csv
Файл завантажено NOAA_ID_5_23-03-2024_08-03-05.csv
Файл завантажено NOAA_ID_6_23-03-2024_08-03-07.csv
Файл завантажено NOAA_ID_7_23-03-2024_08-03-09.csv
Файл завантажено NOAA_ID_8_23-03-2024_08-03-10.csv
Файл завантажено NOAA_ID_9_23-03-2024_08-03-11.csv
Файл завантажено NOAA_ID_10_23-03-2024_08-03-13.csv
Файл завантажено NOAA_ID_11_23-03-2024_08-03-14.csv
Файл завантажено NOAA_ID_12_23-03-2024_08-03-15.csv
Файл завантажено NOAA_ID_13_23-03-2024_08-03-15.csv
Файл завантажено NOAA_ID_14_23-03-2024_08-03-17.csv
Файл завантажено NOAA_ID_15_23-03-2024_08-03-18.csv
Файл завантажено NOAA_ID_16_23-03-2024_08-03-19.csv
Файл завантажено NOAA_ID_17_23-03-2024_08-03-20.csv
Файл завантажено NOAA_ID_18_23-03-2024_08-03-22.csv
Файл завантажено NOAA_ID_19_23-03-2024_08-03-23.csv
Файл завантажено NOAA

#### 3) Зчитати завантажені текстові файли у фрейм (детальніше про роботу із фреймами буде розказано у подальших лабораторних роботах). Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [5]:
def dataframe(path):
    # Створення списку заголовків для DataFrame
    headers = ['Year', 'week', 'SMN', 'SMT', 'VCI', 'TCZ', 'VHI', 'empty']
    # Ініціалізація порожнього списку для зберігання окремих DataFrame
    all_dataframes = []

    # Ітерація по кожному файлу у вказаній директорії
    for filename in os.listdir(path):
        # Перевірка, чи файл є CSV
        if not filename.endswith('.csv'):
            continue
        # Розділення імені файлу на токени за підстрокою "_"
        tokens = filename.split("_")
        # Перевірка, чи файл починається з "NOAA"
        if tokens[0] != "NOAA":
            continue
        # Видобування індексу з імені файлу
        index = tokens[2]
        # Побудова повного шляху до файлу
        filepath = os.path.join(path, filename)
        # Зчитування CSV-файлу в DataFrame, пропускаючи перший рядок і використовуючи задані заголовки
        df = pd.read_csv(filepath, header=1, names=headers) 
        # Видалення рядків, де значення стовпця VHI дорівнює -1
        df = df.drop(df.loc[df['VHI'] == -1].index)
        # Додавання нового стовпця 'Region' з виділеним індексом
        df['Region'] = index
        # Додавання обробленого DataFrame до списку
        all_dataframes.append(df)
 
    # Об'єднання всіх окремих DataFrame в один та скидання індексу
    combined_df = pd.concat(all_dataframes, ignore_index=True).reset_index(drop=True) 
    # Очищення стовпця 'Year' від HTML-тегів
    combined_df["Year"] = combined_df['Year'].str.replace('<tt><pre>', '').str.replace('</pre></tt>', "")
    # Видалення рядків з порожніми значеннями у стовпці 'Year'
    combined_df = combined_df[combined_df["Year"].astype(str).str.strip() != '']
    # Конвертація значень стовпця 'Year' у тип float, а потім у тип int
    combined_df.loc[:, "Year"] = combined_df["Year"].astype(float).astype(int)
    # Конвертація значень стовпця 'Region' у тип int
    combined_df.loc[:, "Region"] = combined_df["Region"].astype(int)
    # Видалення стовпця 'empty'
    combined_df = combined_df.drop(columns=['empty'])
    # Видалення рядків з NaN значеннями
    combined_df = combined_df.dropna()

    return combined_df

# Виклик функції dataframe з вказаним шляхом до директорії
combined_dataframe = dataframe("AD")
# Виведення отриманого комбінованого DataFrame
combined_dataframe



,Year,week,SMN,SMT,VCI,TCZ,VHI,Region
0,1982,1.0,0.058,258.03,51.18,49.16,50.17,10
1,1982,2.0,0.062,261.39,55.47,38.40,46.94,10
2,1982,3.0,0.061,263.35,56.51,32.77,44.64,10
3,1982,4.0,0.058,264.99,52.86,28.74,40.80,10
4,1982,5.0,0.055,266.32,45.90,28.59,37.25,10
...,...,...,...,...,...,...,...,...
57936,2024,7.0,0.121,279.65,61.39,4.56,32.98,9
57937,2024,8.0,0.132,281.83,60.33,4.55,32.44,9
57938,2024,9.0,0.143,283.74,59.19,5.59,32.39,9
57939,2024,10.0,0.152,285.45,56.45,7.94,32.19,9


#### 4) Реалізувати процедуру, яка змінить індекси областей, які використані на порталі NOAA на наступні:
#### № області Назва
##### 1 Вінницька 
##### 2 Волинська 
##### 3 Дніпропетровська 
##### 4 Донецька 
##### 5 Житомирська 
##### 6 Закарпатська 
##### 7 Запорізька 
##### 8 Івано-Франківська 
##### 9 Київська 
##### 10 Кіровоградська 
##### 11 Луганська 
##### 12 Львівська 
##### 13 Миколаївська
##### 14 Одеська
##### 15 Полтавська
##### 16 Рівенська
##### 17 Сумська
##### 18 Тернопільська
##### 19 Харківська
##### 20 Херсонська
##### 21 Хмельницька
##### 22 Черкаська
##### 23 Чернівецька
##### 24 Чернігівська
##### 25 Республіка Крим

In [6]:
combined_dataframe["Region"] = combined_dataframe["Region"].replace({1: 'Вінницька', 2: 'Волинська', 3: 'Дніпропетровська', 4: 'Донецька', 5: 'Житомирська',
                                   6: 'Закарпатська', 7: 'Запорізька', 8: 'Івано-Франківська', 9: 'Київська', 10: 'Кіровоградська',
                                   11: 'Луганська', 12: 'Львівська', 13: 'Миколаївська', 14: 'Одеська', 15: 'Полтавська',
                                   16: 'Рівенська', 17: 'Сумська', 18: 'Тернопільська', 19: 'Харківська', 20: 'Херсонська',
                                   21: 'Хмельницька', 22: 'Черкаська', 23: 'Чернівецька', 24: 'Чернігівська', 25: 'Республіка Крим',
                                   26: 'Київ', 27: 'Севастополь'})
combined_dataframe

,Year,week,SMN,SMT,VCI,TCZ,VHI,Region
0,1982,1.0,0.058,258.03,51.18,49.16,50.17,Кіровоградська
1,1982,2.0,0.062,261.39,55.47,38.40,46.94,Кіровоградська
2,1982,3.0,0.061,263.35,56.51,32.77,44.64,Кіровоградська
3,1982,4.0,0.058,264.99,52.86,28.74,40.80,Кіровоградська
4,1982,5.0,0.055,266.32,45.90,28.59,37.25,Кіровоградська
...,...,...,...,...,...,...,...,...
57936,2024,7.0,0.121,279.65,61.39,4.56,32.98,Київська
57937,2024,8.0,0.132,281.83,60.33,4.55,32.44,Київська
57938,2024,9.0,0.143,283.74,59.19,5.59,32.39,Київська
57939,2024,10.0,0.152,285.45,56.45,7.94,32.19,Київська


#### 5) Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):
##### 5.1) Ряд VHI для області за вказаний рік, пошук екстремумів (min та max);

In [7]:
def find_extremes(df, index, year):
    # Вибираємо дані за певний регіон та рік
    vhi = df[(df["Region"] == index) & (df["Year"] == year)]
    # Знаходимо мінімальне та максимальне значення індексу VHI
    min_vhi = vhi["VHI"].min()
    max_vhi = vhi["VHI"].max()
    # Виводимо інформацію про екстремуми
    print("Область:", index)
    print("Рік:", year)
    print("Мінімальний VHI:", min_vhi)
    print("Максимальний VHI:", max_vhi)

# Викликаємо функцію find_extremes з комбінованим DataFrame та вказаним регіоном та роком
find_extremes(combined_dataframe, 'Київ', 2000)


Область: Київ
Рік: 2000
Мінімальний VHI: 26.51
Максимальний VHI: 59.34


#### 5) Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):
##### 5.2)Ряд VHI за вказаний діапазон років для вказаних областей;
виявити роки, протягом яких екстремальні посухи торкнулися
більше вказаного відсотка областей по Україні (20% областей -
5 областей з 25);
##### 5.3)Аналогічно для помірних посух

In [9]:
def extreme_drought_years(df, regions, start_year, end_year, percent):
    # Для кожного регіону вказаних областей
    for region in regions:
        # Вивести назву області
        print(f"Область {region}:")
        # Проходження по рокам вказаного діапазону
        for year in range(start_year, end_year + 1):
            # Вибірка значень VHI для поточного регіону та року
            vhi = df[(df['Region'] == region) & (df['Year'] == year)]['VHI']
            # Якщо вибірка не порожня
            if not vhi.empty:
                # Вивести значення VHI для поточного року
                print(f"Рік {year}: {vhi.values[0]}")
            else:
                # Якщо вибірка порожня, вивести повідомлення про відсутність даних
                print(f"Немає даних для регіону {region} за рік {year}")
        print("\n")
    
    # Визначення загальної кількості регіонів
    total_regions = df['Region'].nunique()
    extreme_drought_years_years = []
    # Проходження по рокам
    for year in range(df['Year'].min(), df['Year'].max() + 1):
        # Визначення кількості областей, які страждають від екстремальної посухи
        affected_regions = df[(df['Year'] == year) & (df['VHI'] < 20) & (df['VHI'] > 1 - 1)]['Region'].nunique()
        # Якщо кількість областей перевищує відсоток від загальної кількості
        if affected_regions > total_regions * (percent / 100):
            # Додати рік до списку років з екстремальною посухою
            extreme_drought_years_years.append(year)
    # Вивести роки з екстремальною посухою
    print(f"Роки екстримальних посух, які вразили понад {percent}% областей:")
    print(extreme_drought_years_years)

# Виклик функції для пошуку років екстремальної посухи
extreme_drought_years(combined_dataframe, ['Одеська', 'Полтавська', 'Рівенська', 'Сумська', 'Тернопільська'], 1999, 2004, 20)

def moderate_drought_years(df, regions, start_year, end_year, percent):
    # Загальна кількість областей
    total_regions = len(regions)
    moderate_drought_years_years = []
    # Проходження по рокам вказаного діапазону
    for year in range(start_year, end_year + 1):
        # Визначення кількості областей, які страждають від помірної посухи
        affected_regions = df[(df['Year'] == year) & (df['VHI'] < 35) & (df['VHI'] > 20)]['Region'].nunique()
        # Якщо кількість областей перевищує відсоток від загальної кількості
        if affected_regions > total_regions * (percent / 100):
            # Додати рік до списку років з помірною посухою
            moderate_drought_years_years.append(year)
    # Вивести роки з помірною посухою
    print(f"Роки помірних посух, які вразили понад {percent}% областей:")
    print(moderate_drought_years_years)

# Виклик функції для пошуку років помірної посухи
moderate_drought_years(combined_dataframe, ['Одеська', 'Полтавська', 'Рівенська', 'Сумська', 'Тернопільська'], 1999, 2004, 20)


Область Одеська:
Рік 1999: 47.48
Рік 2000: 39.41
Рік 2001: 40.58
Рік 2002: 73.59
Рік 2003: 62.06
Рік 2004: 59.31


Область Полтавська:
Рік 1999: 43.57
Рік 2000: 38.69
Рік 2001: 33.22
Рік 2002: 60.59
Рік 2003: 47.87
Рік 2004: 62.45


Область Рівенська:
Рік 1999: 38.03
Рік 2000: 38.38
Рік 2001: 29.91
Рік 2002: 47.14
Рік 2003: 46.56
Рік 2004: 65.79


Область Сумська:
Рік 1999: 44.86
Рік 2000: 33.99
Рік 2001: 32.36
Рік 2002: 55.08
Рік 2003: 49.91
Рік 2004: 66.01


Область Тернопільська:
Рік 1999: 47.27
Рік 2000: 42.98
Рік 2001: 42.06
Рік 2002: 54.87
Рік 2003: 61.68
Рік 2004: 79.2


Роки екстримальних посух, які вразили понад 20% областей:
[1986, 1992, 1993, 1994, 2000, 2007]
Роки помірних посух, які вразили понад 20% областей:
[1999, 2000, 2001, 2002, 2003, 2004]
